<a href="https://colab.research.google.com/github/silviolima07/Regression_Apartment_Sales/blob/master/Scrap_Imovel_venda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Scrap de informações de venda de apartamentos
### Foram coletadas 4k anúncios a partir da busca em 2k páginas
### A busca foi executada separadamente para cada item
### Pois ao tentar trazer todas simultaneamente, o número
### de linhas por item era 19 e não 20, o valor correto.
### Executando a função para cada item, o processo foi ok.

![alt text](https://drive.google.com/uc?id=1zzEO05kcvFo7dS1oeBVVGg_Ze4By3dCC)

In [ ]:
!pip install bs4 -U

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import re



In [ ]:
# Esta função ira efetuar a pesquisa nas páginas indicadas pela url onde os anúncios estão disponíveis.
# É adicionado o número da página e a pesquisa é refeita, assim posso pesquisar desde 1 até o valor que se deseje.
# De cada página é feito o parser e seu conteúdo pode então ser usado para buscar padrões.
# A busca de padrões foi feita com expressões regulares.
# Primeiro identifica-se através de um inspect no browse a informação deseja e a classe mais próxima onde está localizada.
# Depois disso vai se filtrando e ajustando a expressão regular até conseguir identificar um padrão que pode ser usado na extração.
#
def scrap_imovel_web(n):
  print("Pagina:",n)
 
  url_page = 'https://www.imovelweb.com.br/apartamentos-venda-sao-paulo-sp-mais-de-1-banheiro-mais-de-1-vaga-de-10-a-20-anos-pagina'

  print(url_page+'-'+str(n)+'.html')
  scrap = requests.get(url_page+'-'+str(n)+'.html')
  soup = bs(scrap.content, 'html.parser')
  
  # Classes localizadas através do inspect feito através do browse da página
  class_main_features = soup.find_all(class_ = "main-features")
  class_posting_location = soup.find_all('span',class_ = "posting-location go-to-posting")
  class_first_price = soup.find_all(class_ = "first-price")

  tag_features = class_main_features
  tag_location = class_posting_location
  tag_price = class_first_price

  # Através de testes foram identificados os seguintes padrões para extração das informações desejadas em cada página
  pattern_qto = '[1-90-9 ]*Q[a-z]*' # Quarto ok
  
  pattern_vaga = '[0-9 ]*V[a-z]+' # Vagas ok

  pattern_area = '[0-9][0-9]*[ ]m²[ áa-zú]*' # area util ok        url[0][0:16] = area total  url[1][0:16] = area util
  
  pattern_banheiro = '[0-90-9]* B[a-z]+' # Banheiro ok

  pattern_endereco = '<span>[ :alnum:]+[ [íÍáiÓêA-Za-zaúçíéôáãa-zoiÍ]*' # endereço ok
 
  # Aplicação das expressões regulares para trazer as linhas com informações em cada página pesquisa
  # Cada página tinha 20 linhas de cada informação, esse valor deve ser igual pra todos items.
  quarto = re.findall(pattern_qto, str(tag_features))
  vaga = re.findall(pattern_vaga, str(tag_features))
  banheiro = re.findall(pattern_banheiro, str(tag_features))
  area = re.findall(pattern_area, str(tag_features))
  endereco = re.findall(pattern_endereco, str(tag_location))

  #################################################################################################
  
  # A informação price, precisou de duas etapas, por isso possui dois padrões para aplicação das expressões regulares
  pattern_temp = 'data-price=["R[$ 0-9.]*'  # mostrar price ok
  pattern = 'R[$ 0-9.]*'

  temp = re.findall(pattern_temp, str(tag_price)) 
  price = re.findall(pattern, str(temp))

  #################################################################################################

  # Pode-se aplicar a função e trazer todas informações de uma vez ou separadamente, comentando a linha do return e deixando
  # apenas a variável desejada. Foi feito dessa forma, para garantir que todas viessem em número de 20 por página.
  # Nas tentativas de trazer todas juntas de uma vez, em alguns momentos, vinha 19 para algumas e isso não permitia a criação do
  # dataset final com todas informações agrupadas por página pesquisada
  print("Concluido")
  # banheiro, quarto, vaga , endereco, price , area
  # out1       out2   out3   out4      out5    out6
  return  banheiro #, quarto, vaga , endereco, price , area
  
  



## Identificar os elementos de cada página
### - Visualizar cada página onde foi feito o html.parser antes de preparar as expressões regulares
### - Buscar a informação desejada
### - Gerar as expressões regulares pra extrair informações

In [ ]:
# Padrões de busca
# Fase de Teste de busca
# Esta fase é feita antes da função final do scrap
# Aqui busca-se a expressão regular que será aplicada em cada página

url_page = 'https://www.imovelweb.com.br/apartamentos-venda-sao-paulo-sp-de-10-a-20-anos-pagina-2.html' # página2
#url_page = 'https://www.imovelweb.com.br/apartamentos-venda-sao-paulo-sp-de-10-a-20-anos.html'         # página1

scrap = requests.get(url_page)
soup = bs(scrap.content, 'html.parser')

class_main_features = soup.find_all(class_ = "main-features")
class_posting_location = soup.find_all('span',class_ = "posting-location go-to-posting")
class_first_price = soup.find_all(class_ = "first-price")

# Fazer um print de cada tag pra visualizar conteúdo encontrada quando fez o inspect no browse da página
tag_features = class_main_features
tag_location = class_posting_location
tag_price = class_first_price

#pattern = '[RV][a-z][ "<>ía-zA-Z,0-9ã]*'# class_posting_location ok
#pattern = 'R[$ 0-9.]*'  # mostrar price ok
pattern = '<b>[ a-zA-Z0-9]*'  # 


pattern = '[0-9][0-9][ ]m²[ áa-zú]*'# area util e total ok
pattern = '[0-9 ] Q[a-z]*' # Quarto ok
pattern = '[0-9 ] V[a-z]*' # Vagas ok
pattern = '[0-90-9]* B[a-z]+'# ok Banheiro
pattern = '[0-90-90-9]* m' # area total e area util
pattern = '[0-9][0-9][ ]m²[ área total]+'# area total ok
pattern = '[0-9][0-9]*[ ]m²[ áa-zú]*'# area util ok        url[0][0:16] = area total  url[1][0:16] = area util

#pattern = '<span>[ A-Z][ ía-zA-Z,ã]+'# bairro ok

#pattern = '[AVR]+[ -ÃãA-Z,0-9]*'# local



#pattern = '<span>[ A-Z][ôa-z.A-Z][ "<>ÍíéÉáa-zA-Z,0-9ÇçÃã]*'

#pattern_temp = 'data-price=["R[$ 0-9.]*'  # mostrar price ok

#pattern = 'R[$ 0-9.]*'

#pattern = '[0-90-9]* B[a-z]+'# ok Banheiro
#<li><b>1 Banheiro</b></li>

#pattern = '*[0-9 ] Vaga*' # Vagas ok
#pattern = '[0-9 ]*V[a-z]+'
#pattern2 = '[0-9 a-zA-Z\']+'
pattern = '[RV][a-z][ "<>íáa-zA-Z,0-9ã]+'# class_posting_location ok
pattern = '<span>[ :alnum:]+[ [íÍáiÓêA-Za-zaúçíéôáãa-zoiÍ]*'# class_posting_location ok
#pattern = '[A-Z][:alnum:]+[ÍáiÓêA-Za-zaúçíéôáãa-zoi,Í]*'
#temp = re.findall(pattern_temp, str(tag_price)) 

pattern = '[1-90-9 ]*Q[a-z]*' # Quarto ok

url = re.findall(pattern, str(tag_features)) 
url

['2 Quartos',
 '1 Quarto',
 '2 Quartos',
 '1 Quarto',
 '1 Quarto',
 '2 Quartos',
 '1 Quarto',
 '3 Quartos',
 '2 Quartos',
 '2 Quartos',
 '2 Quartos',
 '1 Quarto',
 '3 Quartos',
 '1 Quarto',
 '1 Quarto',
 '1 Quarto',
 '1 Quarto',
 '1 Quarto',
 '4 Quartos',
 '1 Quarto']

## Teste do scrap de uma página

In [ ]:
# Fazer o scrap nas paginas de 1 a 50
banheiro_1_50 = []
for i in range(1,51):
    out  = scrap_imovel_web(i)
    print(" banheiro:", len(out))
    if len(out) == 19:            # Toda página deve trazer 20 linhas de informações sobre banheiro
        print(out)                # Caso o valor seja 19, interromper o scrap
        exit  
    banheiro_1_50.append((out,i))


Pagina: 1
https://www.imovelweb.com.br/apartamentos-venda-sao-paulo-sp-mais-de-1-banheiro-mais-de-1-vaga-de-10-a-20-anos-pagina-1.html
Concluido
 banheiro: 20
Pagina: 2
https://www.imovelweb.com.br/apartamentos-venda-sao-paulo-sp-mais-de-1-banheiro-mais-de-1-vaga-de-10-a-20-anos-pagina-2.html
Concluido
 banheiro: 20
Pagina: 3
https://www.imovelweb.com.br/apartamentos-venda-sao-paulo-sp-mais-de-1-banheiro-mais-de-1-vaga-de-10-a-20-anos-pagina-3.html
Concluido
 banheiro: 20
Pagina: 4
https://www.imovelweb.com.br/apartamentos-venda-sao-paulo-sp-mais-de-1-banheiro-mais-de-1-vaga-de-10-a-20-anos-pagina-4.html
Concluido
 banheiro: 20
Pagina: 5
https://www.imovelweb.com.br/apartamentos-venda-sao-paulo-sp-mais-de-1-banheiro-mais-de-1-vaga-de-10-a-20-anos-pagina-5.html
Concluido
 banheiro: 20
Pagina: 6
https://www.imovelweb.com.br/apartamentos-venda-sao-paulo-sp-mais-de-1-banheiro-mais-de-1-vaga-de-10-a-20-anos-pagina-6.html
Concluido
 banheiro: 20
Pagina: 7
https://www.imovelweb.com.br/apartam

In [ ]:
len(banheiro_1_50)

50

In [ ]:
banheiro_1_50


[(['1 Banheiro',
   '1 Banheiro',
   '2 Banheiros',
   '1 Banheiro',
   '1 Banheiro',
   '1 Banheiro',
   '1 Banheiro',
   '1 Banheiro',
   '1 Banheiro',
   '1 Banheiro',
   '1 Banheiro',
   '1 Banheiro',
   '2 Banheiros',
   '1 Banheiro',
   '2 Banheiros',
   '2 Banheiros',
   '2 Banheiros',
   '2 Banheiros',
   '6 Banheiros',
   '5 Banheiros'],
  1),
 (['1 Banheiro',
   '1 Banheiro',
   '2 Banheiros',
   '3 Banheiros',
   '1 Banheiro',
   '1 Banheiro',
   '1 Banheiro',
   '2 Banheiros',
   '1 Banheiro',
   '1 Banheiro',
   '1 Banheiro',
   '2 Banheiros',
   '2 Banheiros',
   '1 Banheiro',
   '1 Banheiro',
   '3 Banheiros',
   '1 Banheiro',
   '2 Banheiros',
   '6 Banheiros',
   '2 Banheiros'],
  2),
 (['1 Banheiro',
   '5 Banheiros',
   '4 Banheiros',
   '2 Banheiros',
   '5 Banheiros',
   '1 Banheiro',
   '1 Banheiro',
   '1 Banheiro',
   '1 Banheiro',
   '2 Banheiros',
   '1 Banheiro',
   '1 Banheiro',
   '3 Banheiros',
   '1 Banheiro',
   '1 Banheiro',
   '4 Banheiros',
   '1 Banh

## Coletar informações das páginas do site
### Estes comandos foram executados gerando o scrap de um item apenas, comentando-se os demais itens na chamada da função:
### out1 = scrap_imovel_web(page)

In [ ]:
# Foram criadas listas vazias que irão armazenar as informações obtidas em cada página
#
quarto =[] # lista com valores de quartos
vaga =[] # lista com os valores de vaga
banheiro =[] # lista com os valores de banheiro
area_total =[] # lista com valores da area total
area_util=[] # lista com valores de area util
endereco=[] # lista de bairros
price=[]

# banheiro, quarto, vaga , endereco, price , area
# out1       out2   out3   out4      out5    out6

for page in range(1,201): # Ir da pagina 1 a 500
    #out1, out2, out3, out4, out5, out6 = scrap_imovel_web(page) 
    out1 = scrap_imovel_web(page) # A função ira receber o numero da pagina e trazer informações de banheiro, conforme esta definido na função
    #print("Banheiro:", len(out1)," Quarto:", len(out2), " Vaga:", len(out3), " endereco:", len(out4), "price:", len(out5),"area:", len(out6))
    
    # Nesse caso ja sabemos que cada página deve ter 20 itens
    for i in range(20):                   # out1, out2 e out3 se deu tudo certo, tem o mesmo tamanho, esse valor sera o range de iteração
        banheiro.append(out1[i])                 # a lista hrefs recebera os href contido na posicao de indice 'i' na lista out1
        quarto.append(out2[i])                   # o mesmo para a lista prices que recebera o price em out2
        vaga.append(out3[i])
        endereco.append(out4[i])
        price.append(out5[i])
                            
    for n in range(0,len(out6),2):               # out6 tem 40 linhas, sendo pares igual area total e impares area util
        area_total.append(out6[n][0:16])         # 0, 2, 4, 6, 8, 10,...
        area_util.append(out6[n+1][0:16])        # 1, 3, 5, 7, 9, 11



Pagina: 1
https://www.imovelweb.com.br/apartamentos-venda-sao-paulo-sp-mais-de-1-banheiro-mais-de-1-vaga-de-10-a-20-anos-pagina-1.html
Concluido
Pagina: 2
https://www.imovelweb.com.br/apartamentos-venda-sao-paulo-sp-mais-de-1-banheiro-mais-de-1-vaga-de-10-a-20-anos-pagina-2.html
Concluido
Pagina: 3
https://www.imovelweb.com.br/apartamentos-venda-sao-paulo-sp-mais-de-1-banheiro-mais-de-1-vaga-de-10-a-20-anos-pagina-3.html
Concluido
Pagina: 4
https://www.imovelweb.com.br/apartamentos-venda-sao-paulo-sp-mais-de-1-banheiro-mais-de-1-vaga-de-10-a-20-anos-pagina-4.html
Concluido
Pagina: 5
https://www.imovelweb.com.br/apartamentos-venda-sao-paulo-sp-mais-de-1-banheiro-mais-de-1-vaga-de-10-a-20-anos-pagina-5.html
Concluido
Pagina: 6
https://www.imovelweb.com.br/apartamentos-venda-sao-paulo-sp-mais-de-1-banheiro-mais-de-1-vaga-de-10-a-20-anos-pagina-6.html
Concluido
Pagina: 7
https://www.imovelweb.com.br/apartamentos-venda-sao-paulo-sp-mais-de-1-banheiro-mais-de-1-vaga-de-10-a-20-anos-pagina-7.

In [ ]:
len(banheiro)

1980

## Tentativa de trazer mais informações de cada vez
### O total de informações deveria ser igual
### 100 paginas, 20 linhas = 2000 linhas por item
### Quarto - 2000
### Vaga - 1735
### Banheiro - 1989

In [ ]:
quarto =[] # lista com valores de quartos
vaga =[] # lista com os valores de vaga
banheiro =[] # lista com os valores de banheiro
area_total =[] # lista com valores da area total
area_util=[] # lista com valores de area util
endereco=[] # lista de bairros
price=[]

# quarto , vaga, banheiro, endereco price area
# out1     out2  out3      out4     out5  out6

for page in range(1,10):
    out1, out2, out3 = scrap_imovel_web(page)
    print(len(out1))                          # Verificando o número de linhas naquela página
    for i in range(len(out1)):
        banheiro.append(out1[i])
        quarto.append(out2[i])
        vaga.append(out3[i])


Pagina: 1
https://www.imovelweb.com.br/apartamentos-venda-sao-paulo-sp-mais-de-1-vaga-de-10-a-20-anos-pagina-1.html
Concluido
19
Pagina: 2
https://www.imovelweb.com.br/apartamentos-venda-sao-paulo-sp-mais-de-1-vaga-de-10-a-20-anos-pagina-2.html
Concluido
20
Pagina: 3
https://www.imovelweb.com.br/apartamentos-venda-sao-paulo-sp-mais-de-1-vaga-de-10-a-20-anos-pagina-3.html
Concluido
20
Pagina: 4
https://www.imovelweb.com.br/apartamentos-venda-sao-paulo-sp-mais-de-1-vaga-de-10-a-20-anos-pagina-4.html
Concluido
20
Pagina: 5
https://www.imovelweb.com.br/apartamentos-venda-sao-paulo-sp-mais-de-1-vaga-de-10-a-20-anos-pagina-5.html
Concluido
20
Pagina: 6
https://www.imovelweb.com.br/apartamentos-venda-sao-paulo-sp-mais-de-1-vaga-de-10-a-20-anos-pagina-6.html
Concluido
20
Pagina: 7
https://www.imovelweb.com.br/apartamentos-venda-sao-paulo-sp-mais-de-1-vaga-de-10-a-20-anos-pagina-7.html
Concluido
20
Pagina: 8
https://www.imovelweb.com.br/apartamentos-venda-sao-paulo-sp-mais-de-1-vaga-de-10-a-20-

In [ ]:
print("area_util:", len(area_util), "area_total:", len(area_total), "Quarto:", len(quarto), "Banheiro:", len(banheiro), "Vaga:", len(vaga), "Bairro:", len(endereco), "Price:", len(price))

area_util: 1980 area_total: 1980 Quarto: 1973 Banheiro: 1973 Vaga: 1973 Bairro: 1973 Price: 1973


## Gerando um dataframe
### As listas geradas quando a função de scrap foi executada separadamente para
### cada item, foram usadas para criar um dataframe com 4000 linhas de anúncios.
### Foram coletados de 2000 páginas onde o scrap foi executado.

In [ ]:
import pandas as pd
data = { 'area_total': area_total,
         'area_util': area_util,
         'quarto': quarto,
          'banheiro': banheiro,
          'vaga': vaga,
          'bairro': endereco,
         'price': price}

#data = { 'price': price}         
      
df_vendas = pd.DataFrame(data, columns=['price', 'area_total', 'area_util', 'quarto', 'banheiro', 'vaga', 'bairro'])
df_vendas

,price,area_total,area_util,quarto,banheiro,vaga,bairro
0,R$ 2.450.000,489 m² área tota,217 m² área útil,4 Quartos,6 Banheiros,5 Vagas,<span> Campo Belo
1,R$ 2.250.000,115 m² área tota,115 m² área útil,1 Quarto,1 Banheiro,1 Vaga,<span> Jardins
2,R$ 770.000,41 m² área total,41 m² área útil,2 Quartos,1 Banheiro,1 Vaga,<span> Vila Nova Conceição
3,R$ 320.000,33 m² área total,33 m² área útil,1 Quarto,1 Banheiro,1 Vaga,<span> Moema
4,R$ 2.070.000,173 m² área tota,173 m² área útil,4 Quartos,6 Banheiros,4 Vagas,<span> Perdizes
...,...,...,...,...,...,...,...
3995,R$ 1.678.000,255 m² área tota,160 m² área útil,3 Quartos,5 Banheiros,3 Vagas,<span> Aclimação
3996,R$ 260.000,50 m² área total,50 m² área útil,2 Quartos,1 Banheiro,1 Vaga,<span> Vila Marari
3997,R$ 1.430.000,104 m² área tota,104 m² área útil,3 Quartos,3 Banheiros,3 Vagas,<span> Chácara Santo Antônio
3998,R$ 315.000,80 m² área total,29 m² área útil,1 Quarto,1 Banheiro,1 Vaga,<span> Bela Vista


In [ ]:
df_vendas.to_csv("imovelweb_venda.csv")